# Notebook 3: Transport Mechanisms - STDIO

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 3 of 13  
**Level:** Intermediate  
**Duration:** ~45 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand the STDIO transport mechanism in depth
2. Learn how subprocess communication works
3. Build a local MCP server using STDIO
4. Debug STDIO servers properly (avoid stdout corruption)
5. Configure Claude Desktop to use STDIO servers
6. Understand security implications of local servers

## 1. Understanding STDIO Transport

### What is STDIO?

**STDIO (Standard Input/Output)** is the default transport for local MCP servers:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    STDIO TRANSPORT ARCHITECTURE                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   HOST PROCESS (e.g., Claude Desktop)                                   │
│   ═══════════════════════════════════                                   │
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                         MCP Client                               │   │
│   │                                                                  │   │
│   │    JSON-RPC Request ──────────────► stdin                       │   │
│   │                                        │                         │   │
│   │    stdout ◄────────────────────── JSON-RPC Response             │   │
│   │                                        │                         │   │
│   │    stderr ◄────────────────────── Logs/Debug (separate!)        │   │
│   │                                                                  │   │
│   └────────────────────────────────────────┼─────────────────────────┘   │
│                                            │                            │
│                                   subprocess.Popen()                    │
│                                            │                            │
│   ┌────────────────────────────────────────┼─────────────────────────┐   │
│   │                         MCP SERVER PROCESS                       │   │
│   │                                        │                         │   │
│   │    stdin ────────► Parse Request ─────┘                         │   │
│   │                         │                                        │   │
│   │                         ▼                                        │   │
│   │                   Execute Tool                                   │   │
│   │                         │                                        │   │
│   │    stdout ◄──── Format Response                                 │   │
│   │    stderr ◄──── Log Messages (NEVER stdout!)                    │   │
│   │                                                                  │   │
│   └──────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### STDIO Characteristics

| Aspect | Description |
|--------|-------------|
| **Communication** | stdin for input, stdout for output |
| **Process Model** | Server runs as subprocess of host |
| **Security** | Inherits host's permissions (secure by default) |
| **Scalability** | Single user only (not shareable) |
| **Latency** | Minimal (no network overhead) |
| **Use Case** | Local tools, development, personal data |

### Why Use STDIO?

1. **Zero network configuration** - No ports, no firewalls
2. **Secure by default** - Process isolation, no external access
3. **Simple deployment** - Just run the script
4. **Perfect for local data** - Filesystem, local databases
5. **Ideal for development** - Quick iteration

In [ ]:
# Understanding subprocess communication in Python
import subprocess
import json
import sys

print("STDIO Transport Concepts")
print("=" * 50)
print(f"stdin  : File descriptor 0 - Input from parent process")
print(f"stdout : File descriptor 1 - Output TO parent process (JSON-RPC!)")
print(f"stderr : File descriptor 2 - Logs/debug (separate channel)")
print("\n⚠️ CRITICAL: NEVER print() to stdout in MCP server!")
print("   print() corrupts the JSON-RPC stream and crashes the connection.")

## 2. The Critical Rule: Never Log to STDOUT

This is the **#1 mistake** in STDIO MCP server development:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ⚠️ STDOUT IS SACRED ⚠️                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ❌ WRONG - This BREAKS the connection:                                │
│   ═══════════════════════════════════════                               │
│                                                                          │
│   def my_tool(args):                                                    │
│       print("Processing request...")  # ← CRASHES!                     │
│       result = do_work(args)                                            │
│       print(f"Result: {result}")      # ← CRASHES!                     │
│       return result                                                     │
│                                                                          │
│   What happens:                                                         │
│   stdout: "Processing request...\n{\"jsonrpc\"..."                     │
│           ↑ Corrupted JSON! Client crashes!                             │
│                                                                          │
│   ────────────────────────────────────────────────────────────────────  │
│                                                                          │
│   ✅ CORRECT - Use stderr for logging:                                  │
│   ════════════════════════════════════                                  │
│                                                                          │
│   import sys                                                            │
│                                                                          │
│   def log(message):                                                     │
│       print(message, file=sys.stderr)  # ← Safe!                        │
│                                                                          │
│   def my_tool(args):                                                    │
│       log("Processing request...")     # ← Goes to stderr             │
│       result = do_work(args)                                            │
│       log(f"Result: {result}")         # ← Goes to stderr             │
│       return result                                                     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Safe logging pattern for STDIO servers
import sys
import logging

# Configure logging to stderr
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stderr  # ← Critical: stderr, not stdout!
)

logger = logging.getLogger("mcp_server")

# This is safe in STDIO servers:
logger.info("This goes to stderr - safe!")
logger.debug("Debug info - also safe!")
logger.warning("Warnings too - stderr!")

# Demonstrate the difference
print("\nDemonstrating output channels:")
print("This would corrupt stdout in a real server!", file=sys.stdout)
print("This is safe - goes to stderr", file=sys.stderr)

## 3. Building a Simple STDIO MCP Server

Let's build a minimal STDIO MCP server from scratch to understand the protocol:

In [ ]:
# Minimal STDIO MCP Server (conceptual - for understanding)
# In practice, use FastMCP or the official SDK

MINIMAL_SERVER_CODE = '''
#!/usr/bin/env python3
"""Minimal STDIO MCP Server - For Educational Purposes"""

import sys
import json
from typing import Any

def log(message: str):
    """Log to stderr - NEVER to stdout!"""
    print(f"[SERVER] {message}", file=sys.stderr)

def send_response(response: dict):
    """Send JSON-RPC response to stdout."""
    json_str = json.dumps(response)
    # MCP uses newline-delimited JSON
    sys.stdout.write(json_str + "\\n")
    sys.stdout.flush()

def handle_initialize(request_id: int, params: dict) -> dict:
    """Handle initialize request."""
    log(f"Initialize requested: {params}")
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "result": {
            "protocolVersion": "2025-06-18",
            "capabilities": {
                "tools": {"listChanged": True}
            },
            "serverInfo": {
                "name": "MinimalMCPServer",
                "version": "0.1.0"
            }
        }
    }

def handle_tools_list(request_id: int) -> dict:
    """Handle tools/list request."""
    log("Tools list requested")
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "result": {
            "tools": [
                {
                    "name": "greet",
                    "description": "Greet someone by name",
                    "inputSchema": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "Name to greet"}
                        },
                        "required": ["name"]
                    }
                }
            ]
        }
    }

def handle_tools_call(request_id: int, params: dict) -> dict:
    """Handle tools/call request."""
    tool_name = params.get("name")
    arguments = params.get("arguments", {})
    log(f"Tool call: {tool_name} with args: {arguments}")
    
    if tool_name == "greet":
        name = arguments.get("name", "World")
        result = f"Hello, {name}! 👋"
    else:
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": -32601,
                "message": f"Unknown tool: {tool_name}"
            }
        }
    
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "result": {
            "content": [{"type": "text", "text": result}],
            "isError": False
        }
    }

def main():
    """Main server loop - read from stdin, write to stdout."""
    log("Server starting...")
    
    for line in sys.stdin:
        line = line.strip()
        if not line:
            continue
            
        try:
            request = json.loads(line)
            log(f"Received: {request.get('method')}")
            
            method = request.get("method")
            request_id = request.get("id")
            params = request.get("params", {})
            
            if method == "initialize":
                response = handle_initialize(request_id, params)
            elif method == "initialized":
                # Notification - no response needed
                log("Client initialized")
                continue
            elif method == "tools/list":
                response = handle_tools_list(request_id)
            elif method == "tools/call":
                response = handle_tools_call(request_id, params)
            else:
                response = {
                    "jsonrpc": "2.0",
                    "id": request_id,
                    "error": {
                        "code": -32601,
                        "message": f"Method not found: {method}"
                    }
                }
            
            send_response(response)
            
        except json.JSONDecodeError as e:
            log(f"JSON parse error: {e}")
        except Exception as e:
            log(f"Error: {e}")

if __name__ == "__main__":
    main()
'''

print("Minimal STDIO MCP Server Code:")
print("=" * 60)
print(MINIMAL_SERVER_CODE)

## 4. Using FastMCP for STDIO Servers

In practice, use **FastMCP** - it handles all the protocol details:

In [ ]:
# FastMCP STDIO Server Example
FASTMCP_SERVER_CODE = '''
#!/usr/bin/env python3
"""FastMCP STDIO Server Example

Save as: my_server.py
Run as:  python my_server.py
"""

from fastmcp import FastMCP
import os

# Create MCP server instance
mcp = FastMCP("MyLocalServer")


@mcp.tool()
def greet(name: str) -> str:
    """Greet someone by name.
    
    Args:
        name: The name of the person to greet
    
    Returns:
        A friendly greeting message
    """
    return f"Hello, {name}! Welcome to MCP! 👋"


@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        Sum of a and b
    """
    return a + b


@mcp.tool()
def get_current_directory() -> str:
    """Get the current working directory.
    
    Returns:
        Absolute path of current directory
    """
    return os.getcwd()


if __name__ == "__main__":
    # Run with STDIO transport (default)
    mcp.run(transport="stdio")
'''

print("FastMCP STDIO Server Example:")
print("=" * 60)
print(FASTMCP_SERVER_CODE)

### Key FastMCP Features

| Feature | Description |
|---------|-------------|
| `@mcp.tool()` | Decorator auto-generates JSON Schema from type hints |
| `@mcp.resource()` | Decorator for defining resources |
| `@mcp.prompt()` | Decorator for prompt templates |
| Docstrings | Automatically used for descriptions |
| Type hints | Converted to JSON Schema types |
| Error handling | Exceptions mapped to JSON-RPC errors |

## 5. Configuring Claude Desktop for STDIO

To use your STDIO server with Claude Desktop, configure `claude_desktop_config.json`:

### Windows Configuration

```powershell
# Location of config file
%APPDATA%\Claude\claude_desktop_config.json
```

In [ ]:
# Claude Desktop configuration example
claude_config = {
    "mcpServers": {
        "my-local-server": {
            "command": "python",
            "args": ["C:\\Users\\YourName\\mcp-servers\\my_server.py"],
            "env": {
                "PYTHONUNBUFFERED": "1"  # Important for real-time output
            }
        },
        "filesystem-server": {
            "command": "python",
            "args": [
                "C:\\Users\\YourName\\mcp-servers\\filesystem_server.py",
                "--allowed-dir", "C:\\Users\\YourName\\Documents"
            ]
        },
        "node-server": {
            "command": "node",
            "args": ["C:\\Users\\YourName\\mcp-servers\\my_node_server.js"]
        }
    }
}

print("Claude Desktop Configuration (Windows):")
print("=" * 60)
print(json.dumps(claude_config, indent=2))
print("\n📁 Save to: %APPDATA%\\Claude\\claude_desktop_config.json")

### Configuration Best Practices

```
┌─────────────────────────────────────────────────────────────────────────┐
│              STDIO CONFIGURATION BEST PRACTICES                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ✅ DO                                ❌ DON'T                          │
│   ════                                 ══════                            │
│                                                                          │
│   • Use absolute paths                • Use relative paths              │
│     "C:\\path\\to\\server.py"          "./server.py"                    │
│                                                                          │
│   • Set PYTHONUNBUFFERED=1            • Forget buffering settings       │
│     (ensures real-time output)          (causes delayed responses)      │
│                                                                          │
│   • Test server manually first        • Configure untested servers      │
│     python server.py < test.json        (hard to debug in Claude)       │
│                                                                          │
│   • Use meaningful server names       • Use cryptic names               │
│     "filesystem-documents"              "srv1"                          │
│                                                                          │
│   • Restart Claude after changes      • Expect hot-reload               │
│     (config only read at startup)       (doesn't work for STDIO)        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 6. Testing STDIO Servers

### Method 1: Manual Testing with Echo

In [ ]:
# Test JSON-RPC messages you can use to test manually
test_messages = {
    "initialize": {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2025-06-18",
            "capabilities": {},
            "clientInfo": {"name": "TestClient", "version": "1.0"}
        }
    },
    "initialized": {
        "jsonrpc": "2.0",
        "method": "initialized"
    },
    "tools_list": {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/list",
        "params": {}
    },
    "tools_call": {
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {
            "name": "greet",
            "arguments": {"name": "Alice"}
        }
    }
}

print("Test Messages for Manual STDIO Testing:")
print("=" * 60)
for name, msg in test_messages.items():
    print(f"\n# {name}")
    print(json.dumps(msg))

### Manual Testing in PowerShell

```powershell
# Test your server manually
# Start the server and send messages

# Method 1: Using echo and pipeline
echo '{"jsonrpc":"2.0","id":1,"method":"initialize","params":{"protocolVersion":"2025-06-18","capabilities":{}}}' | python my_server.py

# Method 2: Interactive testing
python my_server.py
# Then type/paste JSON messages and press Enter
```

### Method 2: Using MCP Inspector

The **MCP Inspector** is the official debugging tool:

```powershell
# Install MCP Inspector
npm install -g @modelcontextprotocol/inspector

# Run Inspector with your server
mcp-inspector python my_server.py

# Opens a web UI at http://localhost:6274
# - View available tools, resources, prompts
# - Test tool calls interactively
# - See raw JSON-RPC messages
```

In [ ]:
# Simple test client simulation
import subprocess
import json

def test_stdio_server(server_command: list, messages: list) -> list:
    """
    Test an STDIO server with a sequence of messages.
    
    Args:
        server_command: Command to start server, e.g., ["python", "server.py"]
        messages: List of JSON-RPC messages to send
    
    Returns:
        List of responses from the server
    """
    # This is a conceptual example - would need actual server file to run
    print(f"Would run: {' '.join(server_command)}")
    print(f"Would send {len(messages)} messages")
    
    # Example of what the code would look like:
    # process = subprocess.Popen(
    #     server_command,
    #     stdin=subprocess.PIPE,
    #     stdout=subprocess.PIPE,
    #     stderr=subprocess.PIPE,
    #     text=True
    # )
    # 
    # responses = []
    # for msg in messages:
    #     process.stdin.write(json.dumps(msg) + "\n")
    #     process.stdin.flush()
    #     response = process.stdout.readline()
    #     responses.append(json.loads(response))
    # 
    # return responses
    
    return [{"status": "conceptual_example"}]

# Example usage (conceptual)
test_messages = [
    {"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {}},
    {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
]

test_stdio_server(["python", "my_server.py"], test_messages)

## 7. STDIO Security Considerations

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    STDIO SECURITY MODEL                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   SECURE BY DEFAULT                                                     │
│   ═════════════════                                                     │
│                                                                          │
│   ✅ Server inherits host process permissions                           │
│   ✅ No network exposure (localhost only)                               │
│   ✅ Filesystem sandbox (can limit directories)                         │
│   ✅ Process isolation from other applications                          │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   STILL NEED TO CONSIDER                                                │
│   ═══════════════════════                                               │
│                                                                          │
│   ⚠️ Server has access to user's files (if tool exposes filesystem)    │
│   ⚠️ Server runs with user's privileges (can execute commands)          │
│   ⚠️ Secrets in environment variables are accessible                   │
│   ⚠️ No authentication (anyone who can run the server has access)       │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   BEST PRACTICES                                                        │
│   ══════════════                                                        │
│                                                                          │
│   1. Limit tool capabilities to minimum needed                          │
│   2. Validate all inputs before executing                               │
│   3. Use allowlists for file/directory access                           │
│   4. Log all operations for audit                                       │
│   5. Don't store secrets in server code                                 │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Secure file access pattern
SECURE_FILE_ACCESS = '''
import os
from pathlib import Path

class SecureFileAccess:
    """Secure file access with directory allowlist."""
    
    def __init__(self, allowed_directories: list[str]):
        # Resolve to absolute paths
        self.allowed = [Path(d).resolve() for d in allowed_directories]
    
    def is_path_allowed(self, path: str) -> bool:
        """Check if path is within allowed directories."""
        try:
            resolved = Path(path).resolve()
            return any(
                resolved == allowed or allowed in resolved.parents
                for allowed in self.allowed
            )
        except Exception:
            return False
    
    def read_file(self, path: str) -> str:
        """Read file if path is allowed."""
        if not self.is_path_allowed(path):
            raise PermissionError(f"Access denied: {path}")
        return Path(path).read_text()
    
    def list_directory(self, path: str) -> list[str]:
        """List directory if path is allowed."""
        if not self.is_path_allowed(path):
            raise PermissionError(f"Access denied: {path}")
        return [str(p) for p in Path(path).iterdir()]

# Usage:
# secure_fs = SecureFileAccess(["C:\\Users\\Me\\Documents"])
# content = secure_fs.read_file("C:\\Users\\Me\\Documents\\file.txt")  # OK
# content = secure_fs.read_file("C:\\Windows\\System32\\...")  # DENIED
'''

print("Secure File Access Pattern:")
print("=" * 60)
print(SECURE_FILE_ACCESS)

## 8. Common STDIO Issues and Solutions

| Issue | Cause | Solution |
|-------|-------|----------|
| Server doesn't respond | print() to stdout | Use stderr for logging |
| Connection drops randomly | Unhandled exception | Wrap in try/except, log errors |
| Slow responses | Buffered output | Set PYTHONUNBUFFERED=1 |
| Server not found | Wrong path in config | Use absolute paths |
| Tools not showing | Schema error | Validate JSON Schema |
| Unicode errors | Encoding mismatch | Use UTF-8 everywhere |

In [ ]:
# Robust error handling pattern
ROBUST_ERROR_HANDLING = '''
import sys
import json
import traceback

def log_error(error: Exception):
    """Log error details to stderr."""
    print(f"ERROR: {type(error).__name__}: {error}", file=sys.stderr)
    traceback.print_exc(file=sys.stderr)

def safe_tool_execution(tool_func, arguments: dict) -> dict:
    """Execute tool with error handling."""
    try:
        result = tool_func(**arguments)
        return {
            "content": [{"type": "text", "text": str(result)}],
            "isError": False
        }
    except TypeError as e:
        log_error(e)
        return {
            "content": [{"type": "text", "text": f"Invalid arguments: {e}"}],
            "isError": True
        }
    except PermissionError as e:
        log_error(e)
        return {
            "content": [{"type": "text", "text": f"Permission denied: {e}"}],
            "isError": True
        }
    except Exception as e:
        log_error(e)
        return {
            "content": [{"type": "text", "text": f"Tool execution failed: {e}"}],
            "isError": True
        }
'''

print("Robust Error Handling Pattern:")
print("=" * 60)
print(ROBUST_ERROR_HANDLING)

## 9. Key Takeaways

### STDIO Transport Summary

1. **Communication**: stdin/stdout for JSON-RPC, stderr for logs
2. **Security**: Inherits process permissions, no network exposure
3. **Use Cases**: Local tools, development, personal data
4. **Golden Rule**: NEVER print to stdout (use stderr!)
5. **Testing**: Use MCP Inspector or manual JSON-RPC

### When to Use STDIO

| ✅ Use STDIO For | ❌ Don't Use STDIO For |
|-----------------|---------------------|
| Local filesystem access | Shared services |
| Development/testing | Multi-user access |
| Personal tools | Remote deployment |
| Desktop integration | Cloud hosting |

### Next Steps

In **Notebook 4: Transport Mechanisms - HTTP & SSE**, we'll:
- Build remote MCP servers with HTTP transport
- Implement Server-Sent Events for streaming
- Handle authentication for remote access

## 10. Exercises

### Exercise 1: Create a Simple Calculator Server

Create a FastMCP server with tools for: add, subtract, multiply, divide

In [ ]:
# Exercise 1: Complete this server
CALCULATOR_SERVER = '''
from fastmcp import FastMCP

mcp = FastMCP("Calculator")

@mcp.tool()
def add(a: float, b: float) -> float:
    """Add two numbers."""
    # Your code here
    pass

# Add subtract, multiply, divide tools

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

print("Complete the calculator server:")
print(CALCULATOR_SERVER)

### Exercise 2: Configure Claude Desktop

Write the `claude_desktop_config.json` to:
1. Run your calculator server
2. Set appropriate environment variables
3. Use absolute paths

In [ ]:
# Exercise 2: Create the config
your_config = {
    "mcpServers": {
        "calculator": {
            # Your configuration here
        }
    }
}

print("Your Claude Desktop config:")
print(json.dumps(your_config, indent=2))

### Exercise 3: Debug a Broken Server

Find and fix the bugs in this server code:

In [ ]:
# Exercise 3: Find the bugs!
BUGGY_SERVER = '''
from fastmcp import FastMCP

mcp = FastMCP("BuggyServer")

@mcp.tool()
def process_file(path: str) -> str:
    """Process a file."""
    print(f"Processing {path}...")  # BUG #1: ???
    
    content = open(path).read()  # BUG #2: ???
    
    return content

if __name__ == "__main__":
    print("Server starting")  # BUG #3: ???
    mcp.run()
'''

print("Find and describe the 3 bugs in this server:")
print(BUGGY_SERVER)

# Your answers:
# BUG #1: 
# BUG #2: 
# BUG #3: 

---

## References

- [MCP Transports Documentation](https://modelcontextprotocol.io/docs/learn/transports)
- [FastMCP GitHub](https://github.com/jlowin/fastmcp)
- [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector)
- [Claude Desktop MCP Setup](https://modelcontextprotocol.io/docs/quickstart)

---

**Next Notebook:** [04_Transport_Mechanisms_HTTP_SSE.ipynb](./04_Transport_Mechanisms_HTTP_SSE.ipynb)